## Part 1

In [136]:
import pandas as pd

In [154]:
documents = []
_id = 1

# read in documents
with open('input.txt') as f:
    doc = {}
    for line in f:
        if not line.strip(): # empty line
            doc['_id'] = _id 
            if len(doc) > 0: documents.append(doc)
            doc = {}
            _id += 1
        else:
            chunks = line.split()
            for chunk in chunks:
                key, value = chunk.split(':')
                doc[key] = value
    doc['_id'] = _id 
    if len(doc) > 0: documents.append(doc)
                
print(documents[:3])
print(documents[-3:])
print(len(documents))

[{'eyr': '2024', 'pid': '662406624', 'hcl': '#cfa07d', 'byr': '1947', 'iyr': '2015', 'ecl': 'amb', 'hgt': '150cm', '_id': 1}, {'iyr': '2013', 'byr': '1997', 'hgt': '182cm', 'hcl': '#ceb3a1', 'eyr': '2027', 'ecl': 'gry', 'cid': '102', 'pid': '018128535', '_id': 2}, {'hgt': '61in', 'iyr': '2014', 'pid': '916315544', 'hcl': '#733820', 'ecl': 'oth', '_id': 3}]
[{'byr': '1987', 'hcl': '#7d3b0c', 'iyr': '2018', 'hgt': '180cm', 'ecl': 'blu', 'eyr': '2029', 'pid': '878348021', '_id': 280}, {'hgt': '183cm', 'cid': '98', 'byr': '1953', 'hcl': '#866857', 'eyr': '2021', 'iyr': '2012', 'pid': '158898193', '_id': 281}, {'eyr': '2030', 'pid': '039638764', 'ecl': 'hzl', 'hgt': '190cm', 'byr': '1926', 'cid': '294', 'hcl': '#b6652a', 'iyr': '2017', '_id': 282}]
282


In [138]:
table = pd.DataFrame(documents)
table_optional_cid = table.drop('cid', axis=1)
table_optional_cid[~table_optional_cid.isna().any(axis=1)]

,eyr,pid,hcl,byr,iyr,ecl,hgt,_id
0,2024,662406624,#cfa07d,1947,2015,amb,150cm,1
1,2027,018128535,#ceb3a1,1997,2013,gry,182cm,2
3,2026,726519569,#a97842,1980,2011,grn,184cm,4
4,2027,619743219,#6b5442,1980,2012,grn,176cm,5
5,2020,982796633,#602927,1969,2014,brn,164cm,6
...,...,...,...,...,...,...,...,...
276,2026,836559549,#c0946f,1981,2011,amb,167cm,277
277,2026,111085991,#602927,1920,2011,blu,182cm,278
278,2027,284436132,#6b5442,1929,2010,oth,75in,279
279,2029,878348021,#7d3b0c,1987,2018,blu,180cm,280


## Part 2

In [89]:
table = pd.DataFrame(documents)

table[['hgt', 'hgt_unit']] = table['hgt'].str.extract(r'(\d+)(\D+)', expand=True)
table[['byr', 'iyr', 'eyr', 'hgt']] = table[['byr', 'iyr', 'eyr', 'hgt']].apply(pd.to_numeric)

# drop column cid
table_optional_cid = table.drop('cid', axis=1)
# remove all NaNs
table_rm_na = table_optional_cid[~table_optional_cid.isna().any(axis=1)]
# byr (Birth Year) - four digits; at least 1920 and at most 2002.
table_byr = table_rm_na[(table_rm_na['byr'] >= 1920) & (table_rm_na['byr'] <= 2002)]
# iyr (Issue Year) - four digits; at least 2010 and at most 2020.
table_iyr = table_byr[(table_byr['iyr'] >= 2010) & (table_byr['iyr'] <= 2020)]
# eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
table_eyr = table_iyr[(table_iyr['eyr'] >= 2020) & (table_iyr['eyr'] <= 2030)]
# hgt (Height) - a number followed by either cm or in:
  #   If cm, the number must be at least 150 and at most 193.
  #   If in, the number must be at least 59 and at most 76.
table_hgt = table_eyr[((table_eyr['hgt_unit'] == 'cm') & (table_eyr['hgt'] >= 150) & (table_eyr['hgt'] <= 193)) |
                      ((table_eyr['hgt_unit'] == 'in') & (table_eyr['hgt'] >= 59) & (table_eyr['hgt'] <= 76))]
# hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
table_hcl = table_hgt[table_hgt['hcl'].str.match(r'\#[a-f0-9]{6}')]
# ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
ecl = ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']
table_ecl = table_hcl[table_hcl['ecl'].isin(ecl)]
# pid (Passport ID) - a nine-digit number, including leading zeroes.
table_pid = table_ecl[table_ecl['pid'].str.match(r'[0-9]{9}')]

table_pid

,eyr,pid,hcl,byr,iyr,ecl,hgt,_id,hgt_unit
0,2024.0,662406624,#cfa07d,1947.0,2015.0,amb,150.0,1,cm
1,2027.0,018128535,#ceb3a1,1997.0,2013.0,gry,182.0,2,cm
3,2026.0,726519569,#a97842,1980.0,2011.0,grn,184.0,4,cm
4,2027.0,619743219,#6b5442,1980.0,2012.0,grn,176.0,5,cm
5,2020.0,982796633,#602927,1969.0,2014.0,brn,164.0,6,cm
...,...,...,...,...,...,...,...,...,...
276,2026.0,836559549,#c0946f,1981.0,2011.0,amb,167.0,277,cm
277,2026.0,111085991,#602927,1920.0,2011.0,blu,182.0,278,cm
278,2027.0,284436132,#6b5442,1929.0,2010.0,oth,75.0,279,in
279,2029.0,878348021,#7d3b0c,1987.0,2018.0,blu,180.0,280,cm


In [175]:
table = pd.DataFrame(documents)

table[['hgt', 'hgt_unit']] = table['hgt'].str.extract(r'(\d+)(\D+)', expand=True)

table = table.drop('cid', axis=1)
table['error_reason'] = ''
table.loc[table.isna().any(axis=1), 'error_reason'] += 'Null value '
table.loc[~table['byr'].str.match(r'[0-9]{4}', na=False), 'error_reason'] += 'byr format error '
table.loc[~table['iyr'].str.match(r'[0-9]{4}', na=False), 'error_reason'] += 'iyr format error '
table.loc[~table['eyr'].str.match(r'[0-9]{4}', na=False), 'error_reason'] += 'eyr format error '
table[['byr', 'iyr', 'eyr', 'hgt']] = table[['byr', 'iyr', 'eyr', 'hgt']].apply(pd.to_numeric)
table.loc[table['byr'] < 1920, 'error_reason'] += 'byr < 1920 '
table.loc[table['byr'] > 2002, 'error_reason'] += 'byr > 2002 '
table.loc[table['iyr'] < 2010, 'error_reason'] += 'iyr < 2010 '
table.loc[table['iyr'] > 2020, 'error_reason'] += 'iyr > 2020 '
table.loc[table['eyr'] < 2020, 'error_reason'] += 'eyr < 2020 '
table.loc[table['eyr'] > 2030, 'error_reason'] += 'eyr > 2030 '
table.loc[~table['hgt_unit'].isin(['cm', 'in']), 'error_reason'] += 'hgt unit wrong '
table.loc[(table['hgt_unit'] == 'cm') & (table['hgt'] < 150), 'error_reason'] += 'cm < 150 '
table.loc[(table['hgt_unit'] == 'cm') & (table['hgt'] > 193), 'error_reason'] += 'cm > 193 '
table.loc[(table['hgt_unit'] == 'in') & (table['hgt'] < 59), 'error_reason'] += 'in < 59 '
table.loc[(table['hgt_unit'] == 'in') & (table['hgt'] > 76), 'error_reason'] += 'in > 76 '
table.loc[~table['hcl'].str.match(r'\#[a-f0-9]{6}', na=False), 'error_reason'] += 'hcl format error '
table.loc[~table['ecl'].isin(['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']), 'error_reason'] += 'ecl invalid '
table.loc[~table['pid'].str.match(r'[0-9]{9}', na=False), 'error_reason'] += 'pid format error '
table[table.pid == '2807544665']

,eyr,pid,hcl,byr,iyr,ecl,hgt,_id,hgt_unit,error_reason
96,2021.0,2807544665,#623a2f,1928.0,2013.0,oth,191.0,97,cm,


In [90]:
mask = ~table.reset_index()._id.isin(table_pid._id)
leftovers = table[pd.Series(data=mask.values, index=table.index)]
leftovers = leftovers.drop('cid', axis=1)
print(leftovers[~leftovers.isna().any(axis=1)])

        eyr         pid      hcl     byr     iyr      ecl    hgt  _id hgt_unit
12   2020.0   168786676  #fffffd  2030.0  1936.0  #a4a596  168.0   13       cm
13   1974.0       190cm   7fcaa5  2030.0  2026.0      utc   67.0   14       cm
14   2037.0     #ea9083        z  2023.0  2026.0      lzr   59.0   15       cm
16   2020.0  3509331253  #888785  2022.0  2030.0      gmt  191.0   17       cm
17   1921.0     #e99d09   336498  2009.0  1994.0  #c3d564  162.0   18       cm
..      ...         ...      ...     ...     ...      ...    ...  ...      ...
262  2024.0    39114027   4b5675  2022.0  2026.0      grt   74.0  263       cm
264  2027.0   800367032  #341e13  2006.0  2012.0  #765111  166.0  265       cm
271  2025.0   851189955   bb331b  2004.0  2016.0      amb  187.0  272       cm
273  2028.0     #24c4af        z  1993.0  2023.0      hzl  175.0  274       cm
274  2034.0   389455676  #c0946f  1985.0  2013.0      gry  172.0  275       cm

[76 rows x 9 columns]


In [171]:
import re

with open('input.txt') as file:
    passports = file.read().split('\n\n')

def check_height(x):
    m = re.match(r'^(\d{1,})(cm|in)$', x)
    if m:
        if m[2] == 'cm' and 150 <= int(m[1]) <= 193:
            return True
        elif m[2] == 'in' and 59 <= int(m[1]) <= 76:
            return True
    return False

# Validation rules
V = {
    'byr': lambda x: len(x) == 4 and 1920 <= int(x) <= 2002,
    'iyr': lambda x: len(x) == 4 and 2010 <= int(x) <= 2020,
    'eyr': lambda x: len(x) == 4 and 2020 <= int(x) <= 2030,
    'hgt': check_height,
    'hcl': lambda x: re.match(r'#[a-f0-9]{6}', x),
    'ecl': lambda x: x in ('amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'),
    'pid': lambda x: len(x) == 9 and x.isdigit()
}

part1, part2 = 0, 0
for p in passports:
    d = {}
    xs = p.replace('\n', ' ').split(' ')
    for x in xs:
        try:
            key, value = x.split(':')
            d[key] = value
        except ValueError:
            pass
    if all(prop in d for prop in V):
        part1 += 1
        if all(test(d[prop]) for prop, test in V.items()):
            part2 += 1
            print(d['pid'])

print(part1)
print(part2)

662406624
018128535
726519569
619743219
982796633
322719183
838813262
242570886
568185567
795497164
474303066
075846582
645548969
620857794
735047371
348486702
571149069
160915270
569740130
521705827
662418718
693616099
414295491
229803943
990590217
658484617
056975690
153555359
552331609
934058099
149148750
777786047
610226642
912145128
080715145
208417572
205268145
047489285
756830713
679487194
523515724
696621560
969150085
415209726
115098205
709548547
160053490
717041634
065528723
915063263
490625944
491396731
714591144
397963077
196287205
198467794
282852411
321748597
502619036
842400950
204952666
579385370
278271295
731752614
022724981
927525094
221286943
866607511
720439412
187867283
718692455
083172316
993701676
176967666
792694131
431515059
081575957
880191154
886241926
555911148
517975316
133785752
727880050
513722888
470968353
735589126
620818510
952376140
850136149
480452858
837224515
974518338
235426720
916160791
651231060
086942161
278922687
290981079
559804716
947831563
